# Test Cookie

In [1]:
import requests as rq
import tensorflow.keras.models
import json
import numpy as np
import cv2
import os

## get asp.net_sessionid

In [2]:
resp=rq.get('http://ecardfw.upc.edu.cn:20086/')

In [3]:
cookies={'ASP.NET_SessionId':resp.cookies['ASP.NET_SessionId']}

In [4]:
import time
validate_code_time=str(time.time()).replace('.','')
validate_code_url='http://ecardfw.upc.edu.cn:20086/Login/GetValidateCode?time='.format(validate_code_time)
validate_code_resp=rq.get(validate_code_url,cookies=cookies)

In [5]:
with open('./validate_code_tmp.jpg','wb') as f:
    f.write(validate_code_resp.content)
    f.flush()
    validate_img=cv2.imread(f.name)
os.remove('./validate_code_tmp.jpg')

### all

In [34]:
resp=rq.get('http://ecardfw.upc.edu.cn:20086/')
cookies={'ASP.NET_SessionId':resp.cookies['ASP.NET_SessionId']}
import time
validate_code_time=str(time.time()).replace('.','')
validate_code_url='http://ecardfw.upc.edu.cn:20086/Login/GetValidateCode?time='.format(validate_code_time)
validate_code_resp=rq.get(validate_code_url,cookies=cookies)
with open('./validate_code_tmp.jpg','wb') as f:
    f.write(validate_code_resp.content)
    f.flush()
    validate_img=cv2.imread(f.name)
os.remove('./validate_code_tmp.jpg')

## calculate the calidate code

In [35]:
headers={
    'Host': 'ecardfw.upc.edu.cn:20086',
    'Proxy-Connection': 'keep-alive',
    'Content-Length': '72',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'http://ecardfw.upc.edu.cn:20086',
    'Referer': 'http://ecardfw.upc.edu.cn:20086/',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9,zh-TW;q=0.8,en-US;q=0.7,en;q=0.6,fi;q=0.5',
}

In [6]:
models=tensorflow.keras.models.load_model('./result.h5')

In [36]:
def decode(y):
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([str(x) for x in y])
input_width=109
input_height=41
validate_img=cv2.resize(validate_img,(input_width,input_height)).reshape((1,input_height,input_width,3))
pre=models.predict(validate_img/255)
validate_code=decode(pre)
print(validate_code)

18682


In [37]:
post_data={'sno':'1805010118','pwd':'MDUxNTEw','ValiCode':validate_code,'remember':'1','uclass':'1','json':'true'}
login_resp=rq.post('http://ecardfw.upc.edu.cn:20086/Login/LoginBySnoQuery',data=post_data,cookies=cookies,headers=headers)

In [58]:
import base64
s='051510'
base64.b64encode(s.encode('utf-8')).decode('ascii')

'MDUxNTEw'

In [76]:
a=login_resp.content
a=json.loads(a)
a['IsSucceed'] == '1'

False

In [86]:
a=Exception('login failed','captcha_error')
a.args[1]

'captcha_error'

In [15]:
import requests as rq 
import base64
import cv2.cv2

In [23]:
def get_access_token(client_id,client_secret):
    base_url='https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={}&client_secret={}'
    token_resp = rq.get(base_url.format(client_id,client_secret))
    resp_data = token_resp.json()
    print(resp_data)
    if 'error' not in resp_data:
        return resp_data['access_token']

token = get_access_token('IY0jcA953HSa3yBYOoRGMKGX','M2sOdCIWTDZDapNDBiWoUCdkTfMst9Tj')

{'refresh_token': '25.92860d0365483e74b359cc89570f523b.315360000.1918094203.282335-22823469', 'expires_in': 2592000, 'session_key': '9mzdWBaMgoolf731K1H0QI5TMyZe+5CRZJW7vPOZXJnYJFBB1KBeUKn8es4Zj4TxmN6IvJDI3tKBdMC+mOm8yxgvTwL0+Q==', 'access_token': '24.cfde0c4ae3e7c4f35df33f7a029507c0.2592000.1605326203.282335-22823469', 'scope': 'public vis-ocr_ocr brain_ocr_scope brain_ocr_general brain_ocr_general_basic vis-ocr_business_license brain_ocr_webimage brain_all_scope brain_ocr_idcard brain_ocr_driving_license brain_ocr_vehicle_license vis-ocr_plate_number brain_solution brain_ocr_plate_number brain_ocr_accurate brain_ocr_accurate_basic brain_ocr_receipt brain_ocr_business_license brain_solution_iocr brain_qrcode brain_ocr_handwriting brain_ocr_passport brain_ocr_vat_invoice brain_numbers brain_ocr_business_card brain_ocr_train_ticket brain_ocr_taxi_receipt vis-ocr_household_register vis-ocr_vis-classify_birth_certificate vis-ocr_台湾通行证 vis-ocr_港澳通行证 vis-ocr_机动车购车发票识别 vis-ocr_机动车检验合格证识别 vis

In [13]:
def ocr(img_b64,access_token):
    base_url='https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic?access_token={}'
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    params = {'image': img_b64}
    resp = rq.post(base_url.format(access_token),data = params,headers=headers)
    resp_json  = resp.json()
    if 'words_result' in resp_json:
        return resp_json['words_result'][0]['words']

# ocr(None,token)

In [14]:
with open('./GetValidateCode.jpg','rb') as f:
    b64 = base64.b64encode(f.read())
ocr(b64, token)

'11527'

In [22]:
a = cv2.imread('./GetValidateCode.jpg')
b = cv2.imencode('.jpg',a)[1].tostring()
# base64.b64encode(b)
b64

b'/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABSANoDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3v7PB/wA8Y/8AvkUfZ4P+eMf/AHyKkooAj+zwf88Y/wDvkUfZ4P8AnjH/AN8ipKZH5nzeZt+8du307Z96AE+zwf8APGP/AL5FH2eD/njH/wB8ipKKAIWtYGxmJODngYp32eD/AJ4x/wDfIp7qHRkYZVhgj2oRQiKijCqMAe